In [1]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, create_optimizer
from sklearn.model_selection import train_test_split
from tensorflow.data import Dataset
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import shutil

2024-07-23 09:19:16.610948: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 09:19:16.611068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 09:19:16.751065: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
"""# Ініціалізація TPU
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)"""

'# Ініціалізація TPU\nresolver = tf.distribute.cluster_resolver.TPUClusterResolver()\ntf.config.experimental_connect_to_cluster(resolver)\ntf.tpu.experimental.initialize_tpu_system(resolver)\nstrategy = tf.distribute.TPUStrategy(resolver)'

In [3]:
# Load the dataset
file_path = '/kaggle/input/toxic-dataset/dataset_cut.csv'  # Update the path to your dataset
data = pd.read_csv(file_path)

# Preprocess the dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['comment_text'].tolist(), 
    data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist(), 
    test_size=0.1
)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize function
def tokenize_texts(texts, labels, tokenizer, max_length=512):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])

    return (
        np.array(input_ids),
        np.array(attention_masks),
        np.array(labels)
    )

# Tokenize the datasets
train_input_ids, train_attention_masks, train_labels = tokenize_texts(train_texts, train_labels, tokenizer)
val_input_ids, val_attention_masks, val_labels = tokenize_texts(val_texts, val_labels, tokenizer)



# Create TensorFlow datasets
def create_tf_dataset(input_ids, attention_masks, labels, batch_size=16):
    dataset = Dataset.from_tensor_slices(({'input_ids': input_ids, 'attention_mask': attention_masks}, labels))
    dataset = dataset.shuffle(buffer_size=len(input_ids)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

BATCH_SIZE = 16

train_dataset = create_tf_dataset(train_input_ids, train_attention_masks, train_labels, BATCH_SIZE)
val_dataset = create_tf_dataset(val_input_ids, val_attention_masks, val_labels, BATCH_SIZE)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
"""#  Мітки
y_train = data.iloc[:, 2:]

# Ініціалізація словника для ваг класів
class_weights_dict = {}

# Обчислення ваг для кожного класу
for i, column in enumerate(y_train.columns):
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train[column]), y=y_train[column])
    class_weights_dict[i] = class_weights[1]  # class_weight повертає ваги для обох класів 0 і 1, нас цікавить вага для класу 1"""

"#  Мітки\ny_train = data.iloc[:, 2:]\n\n# Ініціалізація словника для ваг класів\nclass_weights_dict = {}\n\n# Обчислення ваг для кожного класу\nfor i, column in enumerate(y_train.columns):\n    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train[column]), y=y_train[column])\n    class_weights_dict[i] = class_weights[1]  # class_weight повертає ваги для обох класів 0 і 1, нас цікавить вага для класу 1"

In [5]:
EPOCHS = 1

# Load pre-trained model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

# Change the classifier layer to include sigmoid activation
model.classifier.activation = tf.keras.activations.sigmoid

# Optimizer and loss
steps_per_epoch = len(train_texts) // BATCH_SIZE
num_train_steps = steps_per_epoch * EPOCHS
num_warmup_steps = int(num_train_steps * 0.1)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps = num_warmup_steps, num_train_steps=num_train_steps)
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

# Compile model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Suppress AutoGraph warnings
@tf.autograph.experimental.do_not_convert
def train_model():
    
    # Training
    history = model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset)

    # Evaluate the model
    val_loss, val_acc = model.evaluate(val_dataset)
    print(f'Validation loss {val_loss} accuracy {val_acc}')
    

train_model()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


I0000 00:00:1721726486.762791      70 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


63/63 [==============================] - 23s 364ms/step - loss: 0.0687 - accuracy: 0.9970
Validation loss 0.06866981834173203 accuracy 0.996999979019165


In [6]:
# Function to predict on a new comment
def predict_comment(comment, model, tokenizer):
    encoding = tokenizer.encode_plus(
        comment,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='tf',
        truncation=True
    )
    
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    
    preds = model.predict({'input_ids': input_ids, 'attention_mask': attention_mask})[0]
    
    print(f'Predictions shape: {preds.shape}')  # Print the shape of predictions
    
    if preds.shape[0] == 1:
        preds = preds[0]  # Get the first element if it's a batch

    return {
        'toxic': preds[0],
        'severe_toxic': preds[1],
        'obscene': preds[2],
        'threat': preds[3],
        'insult': preds[4],
        'identity_hate': preds[5]
    }

# Example usage
comment = "Your example comment here"
prediction = predict_comment(comment, model, tokenizer)
print(prediction)


1/1 [==============================] - 10s 10s/step
Predictions shape: (1, 6)
{'toxic': 0.024661219, 'severe_toxic': 0.017253617, 'obscene': 0.017474754, 'threat': 0.019488987, 'insult': 0.02230708, 'identity_hate': 0.01656551}


In [7]:
import pickle

# Define the path where the model and tokenizer will be saved
model_save_path = '/kaggle/working/saved_model.pkl'
tokenizer_save_path = '/kaggle/working/saved_tokenizer.pkl'

# Save the trained model using pickle
with open(model_save_path, 'wb') as model_file:
    pickle.dump(model, model_file)

# Save the tokenizer using pickle
with open(tokenizer_save_path, 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)
    
from IPython.display import FileLink # Create a downloadable link
display(FileLink(model_save_path))
display(FileLink(tokenizer_save_path))

/opt/conda/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


/kaggle/working/saved_model.pkl

/kaggle/working/saved_tokenizer.pkl

In [8]:
# Example usage
comment = "many think that he is a piece of shit and behaves like a bitch. but he's actually a nice guy"
prediction = predict_comment(comment, model, tokenizer)
print(prediction)

1/1 [==============================] - 0s 61ms/step
Predictions shape: (1, 6)
{'toxic': 0.90759224, 'severe_toxic': 0.27583417, 'obscene': 0.7968681, 'threat': 0.16328111, 'insult': 0.739866, 'identity_hate': 0.20269291}


In [9]:
from sklearn.metrics import classification_report
# Generate predictions for the validation set and compute classification report
def evaluate_model(model, val_dataset, val_labels):
    predictions = []
    true_labels = []
    
    for batch in val_dataset:
        input_ids = batch[0]['input_ids']
        attention_mask = batch[0]['attention_mask']
        labels = batch[1]
        
        preds = model.predict({'input_ids': input_ids, 'attention_mask': attention_mask})[0]
        
        predictions.extend(preds)
        true_labels.extend(labels.numpy())
    
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)
    
    # Binarize predictions
    predictions = np.where(predictions > 0.5, 1, 0)
    
    # Print classification report
    report = classification_report(true_labels, predictions, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], zero_division=0)
    print(report)

evaluate_model(model, val_dataset, val_labels)

1/1 [==============================] - 0s 210ms/step
               precision    recall  f1-score   support

        toxic       0.88      0.64      0.74       109
 severe_toxic       0.00      0.00      0.00         6
      obscene       0.70      0.83      0.76        48
       threat       0.00      0.00      0.00         1
       insult       0.62      0.76      0.68        41
identity_hate       0.00      0.00      0.00         8

    micro avg       0.75      0.66      0.71       213
    macro avg       0.37      0.37      0.36       213
 weighted avg       0.73      0.66      0.68       213
  samples avg       0.06      0.06      0.06       213

